In [ ]:
import cv2
import dlib

# Pre-trained facial landmark detector
predictor_path = "shape_predictor_68_face_landmarks.dat"  # Download path below

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Load an image (replace with your image path or 0 for webcam)
image = cv2.imread("/home/vivek/projects/facial_analysis/image.jpg") 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = detector(gray)

# Find landmarks for each detected face
for face in faces:
    landmarks = predictor(gray, face)

    # Draw circles on each landmark point
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        cv2.circle(image, (x, y), 4, (0, 255, 0), -1)

# Display the result
cv2.imshow("Facial Landmarks", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
